In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (16.0, 16.0)
plt.rcParams['image.interpolation'] = 'nearest'
%load_ext autoreload
%autoreload 2

In [2]:
from cbir.model import get_model
from cbir.features import extract_raw_features
from cbir.utils import load_image, get_list, load_npy_files, covert_data_format
from cbir.utils import run_feature_processing_pipeline, get_list_set
from cbir.utils import result_ap, result_precision, compute_ap
from cbir.crow import apply_crow_aggregation
from cbir.rmac import apply_rmac_aggregation
from cbir.pwa import PWA, get_discriminative_fm
from cbir.query import compute_cosin_distance, simple_query_expansion

model = get_model()
dataset_img_path='/home/jy/dataset/fabric2'
raw_out_path='/home/jy/results/fabric2/raw'
crow_out_path='/home/jy/results/fabric2/crow'
rmac_out_path='/home/jy/results/fabric2/rmac'
pwa_out_path='/home/jy/results/fabric2/pwa'
gt_file_path = '/home/jy/dataset/gt_fabric2.txt'
#query_img_path = '/home/jy/dataset/fabric2'
query_file_path = '/home/jy/dataset/query_fabric2.txt'
query_out_path='/home/jy/results/fabric2/query'
target_size=(512,512)


Using TensorFlow backend.


In [10]:
dataset_img_list = get_list(dataset_img_path)
print(len(dataset_img_list))

def extract_features(model, img_list, output_path, extract_function = None, save_raw = True,\
                    save_features = True):
    pass
    if len(output_path) > 1:
        raw_out_path, out_path = output_path
    else:
        raw_out_path = output_path[0]
    
    #check dir is exist
    if not os.path.exists(raw_out_path):
        os.makedirs(raw_out_path)
    for i in img_list:
        img = load_image(i,target_size)
        raw_feature = extract_raw_features(model,img)
        if save_raw:
            np.save(os.path.join(raw_out_path,os.path.basename(i).split('.')[0]), raw_feature)
        if save_features:
            cvt_raw_feature = covert_data_format(raw_feature)
            out_feature = extract_function(cvt_raw_feature)
            np.save(os.path.join(out_path,os.path.basename(i).split('.')[0]), out_feature)

#extract raw features
extract_features(model, dataset_img_list, (raw_out_path, ), save_features = False)

200


In [3]:
#query list
def read_annotation(path):
    out_dict = dict()
    if not os.path.exists(path):
        return out_dict
    with open(path) as f:
        for i in f:
            a,b = i.strip().split(' ')
            out_dict[a] = b
    return out_dict
#read query file 
query_dict = read_annotation(query_file_path)
gt_dict = read_annotation(gt_file_path)
print(len(query_dict), len(gt_dict))
def get_train_dict(gt, q):
    out_dict = dict()
    for k, v in gt.items():
        if k in q:
            pass
        else:
            out_dict[k] = v
    return out_dict

train_dict = get_train_dict(gt_dict, query_dict)

50 200


In [4]:
#get discrimnate features
def file_dict_to_list(d, path, ext = '.npy'):
    out_list = list()
    if d is not None and len(d):
        for k, v in sorted(d.items()):
            out_list.append(os.path.join(path,k+ext))
    return out_list
train_file_list = file_dict_to_list(train_dict, raw_out_path)
#print(train_file_list)
train_raw_features, train_list = load_npy_files(train_file_list)
channel_weights = get_discriminative_fm(train_raw_features)

def weighting_features(raw_features, weights,out_path = None, save_features = False):
    pass
    out_features = None
    if out_path is not None and not os.path.exists(out_path):
        os.makedirs(out_path)
    for i,f in enumerate(raw_features):
        cvt_raw_feature = covert_data_format(f)  # C x H x W
        pwa_feature = PWA(cvt_raw_feature, weights)
        if save_features:
            np.save(os.path.join(out_path,os.path.basename(raw_files[i])), pwa_feature)
        if out_features is None:
            out_features = pwa_feature
        else:
            out_features = np.vstack([out_features, pwa_feature])
    return out_features

train_features = weighting_features(train_raw_features, channel_weights)
print("train features shape {}".format(train_features.shape))


successful loading of npy file !!!
train features shape (150, 12288)


In [5]:
query_file_list = file_dict_to_list(query_dict, raw_out_path)
query_raw_features, query_list = load_npy_files(query_file_list)
query_features = weighting_features(query_raw_features, channel_weights)
print("query features shape {}".format(query_features.shape))


successful loading of npy file !!!
query features shape (50, 12288)


In [10]:
#query mAP
from cbir.utils import result_precision

def path_to_list(path_list):
    return [os.basename(i).splitext()[0] for i in path_list]

#query_files = path_to_list(query_file_list)
def query(query, train, gt_dict, topK = 3):
    aps=list()
    for i in range(len(query)):
        idxs, rank_dists, rank_names = compute_cosin_distance([query[i]], 
                                                          train, train_list)
        pos,neg = get_list_set(query_list[i].split('-')[0], gt_dict, delimiter = '-')
        pos.remove(query_list[i])
        #print(pos)
        #ap = result_precision(pos, rank_names[:topK])
        ap = compute_ap(pos,rank_names[:topK])
        #print(query_list[i], rank_names[:5], ap)
        #print(ap)
        aps.append(ap)
    return aps

aps = query(query_features, train_features, gt_dict, topK=3)
print('mAP %f' % (np.array(aps).sum()/len(aps),))


mAP 0.521667


In [11]:
#

for i in range(20):
    aps = query(query_features, train_features, gt_dict, topK=i)
    print('mAP {} {:.3f}'.format(i,np.array(aps).sum()/len(aps),))

mAP 0 0.000
mAP 1 0.227
mAP 2 0.428
mAP 3 0.522
mAP 4 0.559
mAP 5 0.567
mAP 6 0.578
mAP 7 0.584
mAP 8 0.586
mAP 9 0.610
mAP 10 0.618
mAP 11 0.629
mAP 12 0.630
mAP 13 0.638
mAP 14 0.638
mAP 15 0.640
mAP 16 0.640
mAP 17 0.647
mAP 18 0.651
mAP 19 0.651
